# Exploratory Data Analysis of Elo Rating Under Different Time Controls and Chess Opening Outcomes

In [1]:
# DataFrame library
import pandas as pd

# Plotting library
import matplotlib.pyplot as plt

# Statistics plotting library based on matplotlib
import seaborn as sns

# Array library
import numpy as np

# Regular expression library for text parsing and extraction
import re

# Datetime library
import datetime as dt

# Library imports for displaying top chess openings (board visualization)
import io
import chess
import chess.pgn
from IPython.display import display, SVG
import chess.svg
import ipywidgets as widgets

In [2]:
chess_games = pd.read_csv("./data/chess_games.csv")

In [3]:
# Gives us an overview of the combined dataframe
chess_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   url               2592 non-null   object 
 1   pgn               2592 non-null   object 
 2   time_control      2592 non-null   object 
 3   end_time          2592 non-null   int64  
 4   rated             2592 non-null   bool   
 5   tcn               2592 non-null   object 
 6   uuid              2592 non-null   object 
 7   initial_setup     2592 non-null   object 
 8   fen               2592 non-null   object 
 9   time_class        2592 non-null   object 
 10  rules             2592 non-null   object 
 11  eco               2591 non-null   object 
 12  white.rating      2592 non-null   int64  
 13  white.result      2592 non-null   object 
 14  white.@id         2592 non-null   object 
 15  white.username    2592 non-null   object 
 16  white.uuid        2592 non-null   object 


In [4]:
chess_games["start_time"] = pd.to_datetime(chess_games["start_time"], unit = "s")
chess_games["end_time"] = pd.to_datetime(chess_games["end_time"], unit = "s")

In [5]:
chess_games.describe()

,end_time,white.rating,black.rating,start_time,accuracies.white,accuracies.black
count,2592,2592.000000,2592.000000,68,1477.000000,1477.000000
mean,2024-08-12 18:48:42.732252928,619.241512,618.815972,2024-05-12 06:33:41.397058816,70.748849,69.531435
min,2023-01-18 02:08:39,100.000000,100.000000,2023-01-19 01:15:56,26.880000,18.410000
25%,2024-03-05 22:30:15,422.000000,427.000000,2023-12-31 15:36:18,63.330000,62.520000
50%,2024-08-29 02:05:29,626.000000,626.500000,2024-04-16 20:13:45.500000,71.350000,70.510000
75%,2025-02-27 18:51:00.249999872,822.250000,821.000000,2024-09-03 00:29:16.500000,79.370000,77.710000
max,2025-06-19 04:38:37,1509.000000,1501.000000,2025-05-27 19:02:43,100.000000,100.000000
std,NaN,231.591152,231.069772,NaN,12.308309,12.495977


In [6]:
# The columns currently in the dataframe
chess_games.columns

Index(['url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn', 'uuid',
       'initial_setup', 'fen', 'time_class', 'rules', 'eco', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'start_time', 'accuracies.white', 'accuracies.black',
       'tournament'],
      dtype='object')

In [7]:
columns_to_drop = ["time_control", "tournament", "rated", "tcn", "uuid", "initial_setup", "accuracies.white", "accuracies.black", 
                   "fen", "white.@id", "white.uuid", "black.@id", "black.uuid"]
chess_games.drop(columns = columns_to_drop, inplace = True)

In [8]:
# Types of games that were played
chess_games["time_class"].unique()

array(['blitz', 'bullet', 'daily', 'rapid'], dtype=object)

In [9]:
# Unique results for white and black (it will be the same results regardless of what column gets picked)
print(np.sort(chess_games["white.result"].unique()))

['abandoned' 'agreed' 'checkmated' 'insufficient' 'repetition' 'resigned'
 'stalemate' 'timeout' 'timevsinsufficient' 'win']


In [10]:
def game_result(df):
    """Determines whether a game was won, lost, or drawn and puts it in a list. 
    The intention is to create another column that can list the result per game regardless of what color was played.

    Args:
        df (pandas.DataFrame): the data that contains all the chess games

    Returns:
        list : Returns a list of outcomes that are one of the three states (Won, Lost, or Draw)
    """
    results = []
    for white_result, white_username, black_result, black_username in zip(df["white.result"], df["white.username"], df["black.result"], df["black.username"]):
        if (white_result == "win" and white_username == "hfactor13") or (black_result == "win" and black_username == "hfactor13"):
            results.append("Won")
        elif white_result == black_result:
            results.append("Draw")
        else:
            results.append("Lost")
    return results

In [11]:
chess_games["game_result"] = game_result(chess_games) 

In [12]:
def my_rating(df):
    """This aggregates rating for all time regardless of the colored piece played.

    Args:
        df (pandas.DataFrame): the data that contains all the chess games

    Returns:
        list: Returns a list of ratings
    """
    rating = []
    for white_rating, black_rating, white_username, black_username in zip(df["white.rating"], df["black.rating"], df["white.username"], df["black.username"]):
        if white_username == "hfactor13":
            rating.append(white_rating)
        elif black_username == "hfactor13":
            rating.append(black_rating)
    return rating

In [13]:
chess_games["my_rating"] = my_rating(chess_games)

In [14]:
# Writes out PGN data to a file (this will be helpful to extract missing information that wasn't already available in the API)
pgn_data = chess_games["pgn"]
with open("./data/pgn_data.pgn", "w") as p:
    p.writelines(pgn_data + "\n")

In [15]:
# Reads in the data from the newly created PGN file to extract information (i.e. the start and end times and the dates)
with open("./data/pgn_data.pgn", "r") as p:
    pgn_text = p.readlines()
pgn_text[:24]

['[Event "Live Chess"]\n',
 '[Site "Chess.com"]\n',
 '[Date "2023.01.18"]\n',
 '[Round "-"]\n',
 '[White "hfactor13"]\n',
 '[Black "MrEvi10verlord"]\n',
 '[Result "0-1"]\n',
 '[CurrentPosition "r6k/ppp4p/2n1r3/8/1nPp4/1P3p2/P1q5/RK4R1 w - -"]\n',
 '[Timezone "UTC"]\n',
 '[ECO "A01"]\n',
 '[ECOUrl "https://www.chess.com/openings/Nimzowitsch-Larsen-Attack-Modern-Variation"]\n',
 '[UTCDate "2023.01.18"]\n',
 '[UTCTime "02:03:52"]\n',
 '[WhiteElo "660"]\n',
 '[BlackElo "795"]\n',
 '[TimeControl "180"]\n',
 '[Termination "MrEvi10verlord won by checkmate"]\n',
 '[StartTime "02:03:52"]\n',
 '[EndDate "2023.01.18"]\n',
 '[EndTime "02:08:39"]\n',
 '[Link "https://www.chess.com/game/live/67746611751"]\n',
 '\n',
 '1. b3 {[%clk 0:02:55.3]} 1... e5 {[%clk 0:02:59.8]} 2. Nf3 {[%clk 0:02:50.5]} 2... Nc6 {[%clk 0:02:58]} 3. g3 {[%clk 0:02:48.6]} 3... d5 {[%clk 0:02:57.8]} 4. c4 {[%clk 0:02:42.1]} 4... Bc5 {[%clk 0:02:54.4]} 5. Bb2 {[%clk 0:02:35.1]} 5... f6 {[%clk 0:02:46.2]} 6. h4 {[%clk 0:02:27.3]}

In [16]:
# Parses text for dates, times, and moves
moves = []
start_times_info = []
start_dates_info = []

for text in pgn_text:
    if re.match("\[StartTime", text):
        start_times_info.append(text)
    elif re.match("\[Date", text):
        start_dates_info.append(text)
    elif re.match("1\.", text):
        moves.append(text)

In [17]:
# Looking at the last 50 characters to determine the total number of moves in the game
last_moves = [m[-50:] for m in moves]
last_moves[:10]

['clk 0:00:07.4]} 29... Qc2# {[%clk 0:01:20.2]} 0-1\n',
 ' {[%clk 0:00:21.2]} 21. g3 {[%clk 0:00:45.3]} 1-0\n',
 '[%clk 0:00:01.1]} 25... h6 {[%clk 0:00:09.9]} 0-1\n',
 '[%clk 0:00:55.3]} 32. Rxh4 {[%clk 0:02:39.7]} 1-0\n',
 '%clk 0:00:01.4]} 26... Qb7 {[%clk 0:00:04.6]} 0-1\n',
 'clk 0:00:12.9]} 17... Qxb4 {[%clk 0:00:01.8]} 0-1\n',
 '3 {[%clk 0:02:17]} 37. Rf4 {[%clk 0:03:18.1]} 1-0\n',
 '[%clk 0:04:05.6]} 39. Qe6# {[%clk 0:04:14.9]} 1-0\n',
 '%clk 0:00:00.4]} 40... Rg6 {[%clk 0:04:13.8]} 0-1\n',
 '[%clk 0:00:05.3]} 21. Nxd4 {[%clk 0:00:13.9]} 1-0\n']

In [18]:
# Reduces the string length so that it starts with '}' and end with '.'
total_moves = [re.findall("\} \d+\.", l)[0] for l in last_moves]

# Extracts the digits which signify the total number of moves
total_moves = [int(re.findall("\d+", total)[0]) for total in total_moves]
total_moves[:10]

[29, 21, 25, 32, 26, 17, 37, 39, 40, 21]

In [19]:
start_dates_series = pd.to_datetime([re.findall("\d+\.\d+\.\d+", s)[0] for s in start_dates_info])
start_times_series = pd.to_timedelta([re.findall("\d+:\d+:\d+", s)[0] for s in start_times_info])
chess_games["start_time"] = start_dates_series + start_times_series

In [20]:
# Add the newly extracted information to the combined games dataframe
chess_games["eco"] = chess_games["eco"].apply(lambda x: str(x)[31:])
chess_games.rename(columns = {"eco": "openings", "time_class": "time_control"}, inplace = True)
chess_games["total_moves"] = total_moves

In [21]:
# Filters out games where there were possibly interruptions or just disconnected entirely
filtered_idx = (chess_games["total_moves"] > 5) & (chess_games["white.result"] != "abandoned") & (chess_games["black.result"] != "abandoned")
chess_games = chess_games[filtered_idx]
chess_games.head()

,url,pgn,end_time,time_control,rules,openings,white.rating,white.result,white.username,black.rating,black.result,black.username,start_time,game_result,my_rating,total_moves
0,https://www.chess.com/game/live/67746611751,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",2023-01-18 02:08:39,blitz,chess,Nimzowitsch-Larsen-Attack-Modern-Variation,660,checkmated,hfactor13,795,win,MrEvi10verlord,2023-01-18 02:03:52,Lost,660,29
1,https://www.chess.com/game/live/67746710345,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",2023-01-18 02:14:48,blitz,chess,Modern-Defense-with-1-e4-2.d4-Bg7-3.Be3,618,win,YasssQueen9,520,timeout,hfactor13,2023-01-18 02:09:23,Lost,520,21
2,https://www.chess.com/game/live/67828846779,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",2023-01-19 00:57:40,bullet,chess,Nimzowitsch-Larsen-Attack-Dutch-Variation,398,timeout,hfactor13,572,win,rico24432,2023-01-19 00:55:37,Lost,398,25
3,https://www.chess.com/game/live/67828893903,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",2023-01-19 01:05:01,blitz,chess,Reti-Opening-Kingside-Fianchetto-Variation,555,win,YusufAwaisu,437,resigned,hfactor13,2023-01-19 00:58:08,Lost,437,32
4,https://www.chess.com/game/live/67829440503,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",2023-01-19 01:15:26,blitz,chess,Nimzowitsch-Larsen-Attack-Classical-Variation-...,348,timeout,hfactor13,367,win,DamianCapponi,2023-01-19 01:05:17,Lost,348,26


In [22]:
# Notice that there's no result called "abandoned" anymore
chess_games["white.result"].unique()

array(['checkmated', 'win', 'timeout', 'resigned', 'stalemate',
       'timevsinsufficient', 'insufficient', 'repetition', 'agreed'],
      dtype=object)

In [23]:
# Number of unique openings
chess_games["openings"].nunique()

844

In [24]:
# Maximum number of openings played for all time controls
max_num_openings_played = chess_games.groupby("openings").size().max()
max_num_openings_played

np.int64(55)

In [25]:
def plot_game_result(df):
    """Creates a dropdown menu for selecting time control and plots the number of games won, lost, or drawn.

    Args:
        df (pandas.DataFrame): the data that contains the chess games
    """
    time_controls = df["time_control"].unique()

    def plot_for_selected_time_control(selected_time_control):
        filtered_df = df[df["time_control"] == selected_time_control]
        print(filtered_df.groupby("game_result").size())
        ax = filtered_df.groupby("game_result").size().plot(kind="bar", title=f"{selected_time_control} Time Control")
        ax.set_yticks([int(ticks) for ticks in ax.get_yticks()])
        ax.set_ylabel("Number of Games Played")
        plt.show()

    widgets.interact(plot_for_selected_time_control, selected_time_control=widgets.Dropdown(options=time_controls, description='Time Control:'))

In [26]:
plot_game_result(chess_games)

interactive(children=(Dropdown(description='Time Control:', options=('blitz', 'bullet', 'daily', 'rapid'), val…

In [27]:
def plot_rating_over_time(df):
    """Creates a dropdown menu for selecting time control and plots rating over time for the selected time control."""
    time_controls = df["time_control"].unique()

    def plot_for_selected_time_control(selected_time_control):
        (df[df["time_control"] == selected_time_control].resample(rule = "D", on = "start_time")
        .mean(numeric_only = True)["my_rating"].interpolate(method = "time")
        .plot(title = f"Rating Over Time ({selected_time_control})", grid = True))
        plt.xlabel("Date")
        plt.ylabel("Rating")
        plt.show()

    widgets.interact(plot_for_selected_time_control, selected_time_control=widgets.Dropdown(options=time_controls, description='Time Control:'))


In [28]:
plot_rating_over_time(chess_games)

interactive(children=(Dropdown(description='Time Control:', options=('blitz', 'bullet', 'daily', 'rapid'), val…

In [29]:
def plot_openings_by_time_control(df):
    """Creates a dropdown menu for selecting time control and plots the top 10 openings for the selected time control.

    Args:
        df (pandas.DataFrame): DataFrame containing chess games
    """
    time_controls = df["time_control"].unique()

    def plot_for_selected_time_control(selected_time_control):
        openings = df[df["time_control"] == selected_time_control].groupby("openings").size().sort_values(ascending=False)
        openings[:10].plot(kind="barh", title=f"Top 10 Openings Played For {selected_time_control.title()} Time Control", xlabel="Number of Games Played")
        plt.xticks([i for i in range(openings.max() + 1)])
        plt.show()

    widgets.interact(plot_for_selected_time_control, selected_time_control=widgets.Dropdown(options=time_controls, description='Time Control:'))

In [30]:
# Top 10 openings played for daily time control
plot_openings_by_time_control(chess_games)

interactive(children=(Dropdown(description='Time Control:', options=('blitz', 'bullet', 'daily', 'rapid'), val…

In [31]:
def plot_openings_matrix(df, n = 10):
    """Creates a dropdown menu for selecting time control and plots the top N openings by game outcome.

    Args:
        df (pandas.DataFrame): DataFrame containing chess games
        n (int): number of openings to display
    """
    time_controls = df["time_control"].unique()

    def plot_for_selected_time_control(selected_time_control):
        filtered_df = df[df["time_control"] == selected_time_control]
        openings_results = (
            filtered_df.groupby(["openings", "game_result"])
            .size()
            .unstack(fill_value=0)
            .sort_values(by="Won", ascending=False)
        )
        ax = sns.heatmap(openings_results[:n], annot=True, cmap='jet')
        cbar = ax.collections[0].colorbar
        cbar.set_ticks([int(tick) for tick in cbar.get_ticks()])
        ax.set_title(f"Top {n} Openings Played By Result For {selected_time_control.title()} Time Control")
        plt.show()

    widgets.interact(plot_for_selected_time_control, selected_time_control=widgets.Dropdown(options=time_controls, description='Time Control:'))

In [32]:
plot_openings_matrix(chess_games)

interactive(children=(Dropdown(description='Time Control:', options=('blitz', 'bullet', 'daily', 'rapid'), val…

In [33]:
def display_top_openings_boards(df, top_n = 10):
    # Get unique time controls
    time_controls = df["time_control"].unique()

    def update_time_control(selected_time_control):
        # Get the top N most played openings for the selected time control
        top_openings = (
            df[df["time_control"] == selected_time_control]
            .groupby("openings")
            .size()
            .sort_values(ascending=False)
            .head(top_n)
            .index
        )

        def show_board(opening):
            # Select the first game with the selected opening and time control
            game_row = df[(df["time_control"] == selected_time_control) & (df["openings"] == opening)].iloc[0]
            pgn_str = game_row["pgn"]
            pgn_io = chess.pgn.read_game(io.StringIO(pgn_str))
            board = pgn_io.board()
            # Attempt to estimate how many moves to play to reach the opening position
            opening_moves = opening.split('-')
            last_move = None
            for part in reversed(opening_moves):
                if any(char.isdigit() for char in part):
                    last_move = part
                    break
            if last_move is not None:
                move_number = int(re.match(r'(\d+)', last_move).group(1))
                ply = move_number * 2 if '...' in last_move else move_number * 2 - 1
            else:
                ply = 4  # Default to 4 plies if move number not found
            for i, move in enumerate(pgn_io.mainline_moves()):
                if i >= ply:
                    break
                board.push(move)
            print(f"Opening: {opening}")
            display(SVG(chess.svg.board(board=board)))

        dropdown = widgets.Dropdown(options=top_openings, description='Opening:')
        widgets.interact(show_board, opening=dropdown)

    widgets.interact(update_time_control, selected_time_control=widgets.Dropdown(options=time_controls, description='Time Control:'))

In [34]:
display_top_openings_boards(chess_games)

interactive(children=(Dropdown(description='Time Control:', options=('blitz', 'bullet', 'daily', 'rapid'), val…